In [ ]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
import subprocess
import gc
from datetime import datetime
from tqdm import tqdm

# -------- SETTINGS ---------
# Choose model source: "dataset" or "huggingface"
MODEL_SOURCE = os.getenv("MODEL_SOURCE", "huggingface")  # dataset or huggingface
HF_TOKEN = os.getenv("HF_TOKEN", None)

if MODEL_SOURCE == "dataset":
    # Model paths from Kaggle datasets
    DIFFUSION_MODEL = "/kaggle/input/flux1-schnell-q4-zip/flux1-schnell-Q4_0.gguf"
    VAE_MODEL = "/kaggle/input/vae-zip/ae.safetensors"
    CLIP_L_MODEL = "/kaggle/input/clip-l-zip/clip_l.safetensors"
    T5XXL_MODEL = "/kaggle/input/t5xxl-zip/t5xxl_fp8_e4m3fn.safetensors"
else:
    # Download from HuggingFace
    from huggingface_hub import hf_hub_download
    print("Downloading models from HuggingFace...")
    
    DIFFUSION_MODEL = hf_hub_download(
        repo_id="city96/FLUX.1-schnell-gguf",
        filename="flux1-schnell-Q4_0.gguf",
        token=HF_TOKEN
    )
    VAE_MODEL = hf_hub_download(
        repo_id="black-forest-labs/FLUX.1-schnell",
        filename="ae.safetensors",
        token=HF_TOKEN
    )
    CLIP_L_MODEL = hf_hub_download(
        repo_id="comfyanonymous/flux_text_encoders",
        filename="clip_l.safetensors",
        token=HF_TOKEN
    )
    T5XXL_MODEL = hf_hub_download(
        repo_id="comfyanonymous/flux_text_encoders",
        filename="t5xxl_fp8_e4m3fn.safetensors",
        token=HF_TOKEN
    )
    print("Models downloaded successfully!")

# -------- PARAMETERS (can be overridden when running via papermill) ---------
PROMPTS = ['A beautiful landscape']
NEGATIVE_PROMPT = ""
OUTPUT_DIR = "images"
IMG_SIZE = (112, 112)
GUIDANCE = 3.5
STEPS = 4
SEED = 42
PRECISION = "q4"
# ----------------------------

print(f"Device: GPU (FLUX GGUF Q4)")
print(f"Model Source: {MODEL_SOURCE}")
print(f"\nPrompts: {len(PROMPTS)}")
print(f"Precision: {PRECISION}")
print(f"Steps: {STEPS}")
print(f"Image size: {IMG_SIZE}")
print(f"Output directory: {OUTPUT_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Setup stable-diffusion.cpp executable
!mkdir -p /kaggle/working/stable-diffusion.cpp
!cp -r /kaggle/input/sd-build-zip/kaggle/working/export/sd_build/* /kaggle/working/stable-diffusion.cpp/
!chmod +x /kaggle/working/stable-diffusion.cpp/build/bin/sd

sd_executable = "/kaggle/working/stable-diffusion.cpp/build/bin/sd"
if not os.path.isfile(sd_executable):
    raise FileNotFoundError("sd executable not found at: " + sd_executable)

print(f"SD executable ready: {sd_executable}")

In [ ]:
# Generate images for each prompt
print("-"*60)
print(f"Model paths:")
print(f"  Diffusion: {DIFFUSION_MODEL}")
print(f"  VAE: {VAE_MODEL}")
print(f"  CLIP-L: {CLIP_L_MODEL}")
print(f"  T5XXL: {T5XXL_MODEL}")
print("-"*60)

for i, prompt in enumerate(tqdm(PROMPTS, desc="Generating")):
    print("-"*60)
    print(f"Prompt {i+1}/{len(PROMPTS)}: {prompt[:60]}...")
    print("-"*60)
    
    # Generate unique filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_{i+1}_{timestamp}.png"
    output_path = os.path.join(OUTPUT_DIR, filename)
    
    cmd = [
        sd_executable,
        "--diffusion-model", DIFFUSION_MODEL,
        "--vae", VAE_MODEL,
        "--clip_l", CLIP_L_MODEL,
        "--t5xxl", T5XXL_MODEL,
        "--prompt", prompt,
        "--cfg-scale", str(GUIDANCE),
        "--sampling-method", "euler",
        "--steps", str(STEPS),
        "--width", str(IMG_SIZE[1]),
        "--height", str(IMG_SIZE[0]),
        "--seed", str(SEED),
        "--output", output_path,
        "--rng", "cuda",
        "-v",
    ]
    
    print(f"Running generation...")
    res = subprocess.run(cmd, capture_output=True, text=True)
    
    print(f"Return code: {res.returncode}")
    if res.returncode == 0:
        print(f"Saved: {output_path}")
    else:
        print(f"ERROR: Generation failed")
        print(f"STDOUT: {res.stdout}")
        print(f"STDERR: {res.stderr}")
    
    # Cleanup
    gc.collect()

print("-"*60)
print(f"COMPLETE! {len(PROMPTS)} images saved to {OUTPUT_DIR}")
print("-"*60)